<a href="https://colab.research.google.com/github/pritijain27/prescriptive-analytics/blob/main/glove_manufacturing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Glove Manufacturing


Install amplpy, pandas and other packages.

In [1]:
!pip install -q amplpy ampltools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.9 MB/s eta 0:00:00


Setup AMPL and select solvers.

In [ ]:
# Google Colab & AMPL integration
MODULES, LICENSE_UUID = ["coin", 'gurobi', "cplex", "highs", "gokestrel"], "42fc7eb6-69aa-445d-b655-3ad24d836541"
from amplpy import tools
from ampltools import cloud_platform_name, ampl_notebook, register_magics

# instantiate AMPL object and register magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = tools.ampl_notebook(modules=MODULES, license_uuid=LICENSE_UUID, g=globals())

register_magics(ampl_object=ampl)

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Read data

In [ ]:
import pandas as pd

process_data = pd.read_excel('/content/drive/MyDrive/645/glove_manufacturing.xlsx',
                             sheet_name="process",
                             index_col=0)

M = list(process_data.index)
P = list(process_data)
print(P,M)
process = process_data.stack().to_dict()
print(process)


Define model.

In [ ]:
ampl.eval ('''

reset;
set M;

param medium_value {i in M} ;
param total_budget ;
param min_pct ;
param min_ratio_news_to_radio ;

var x {i in M} >= 0;


maximize total_value_objective: sum{i in M} medium_value[i]*x[i];

subject to
budget_constraint: sum{i in M} x[i] <= total_budget;
min_pct_constraint {i in M}:x[i] >= min_pct*total_budget;
min_ratio_constraint: x['newspaper'] >= min_ratio_news_to_radio* x['radio'];

''')

Provide data to the model.

In [ ]:
ampl.set['M'] = M

ampl.param['medium_value'] = medium_value
ampl.param['total_budget'] = total_budget
ampl.param['min_pct'] = min_pct
ampl.param['min_ratio_news_to_radio'] = min_ratio_news_to_radio



Display problem formulation.

In [ ]:
ampl.eval('''expand;''')
#ampl.eval('''expand x;''')

Set solver and solve.

In [ ]:
ampl.setOption('solver', 'cplex')
ampl.solve()

Print solution and results.

In [ ]:
# Print results
obj = ampl.getObjective('profit_objective')
print("\n")
print("Total profit is: ", obj.get().value(), "\n")
print("Units Produced:")
ampl.display('x');

The optimal solution is to produce 1.8 gallons of peach lemonade and 1.6 gallons of raspberry lemonade for a profit of \$33.60.